## Summary
This file is for coding to clean and visualize the data to prepare for the unsupervised learning.

In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import miceforest as mf
data_abm = pd.read_csv('abm.csv')
data_card = pd.read_csv('card.csv')
data_eft = pd.read_csv('eft.csv')
data_emt = pd.read_csv('emt.csv')
data_kyc = pd.read_csv('kyc.csv')
data_wire = pd.read_csv('wire.csv')
data_kyc_industry_codes = pd.read_csv('kyc_industry_codes.csv')

# Clean the dataset
## Null value
We first count null value each column.


In [21]:

def replace_other_with_null(dataset):
    dataset = dataset.replace('other', np.nan)  # Replace 'other' with NaN
    return dataset
# Apply the function to each dataset
data_abm = replace_other_with_null(data_abm)
data_card = replace_other_with_null(data_card)
data_kyc = replace_other_with_null(data_kyc)
data_kyc_industry_codes = replace_other_with_null(data_kyc_industry_codes)
data_emt = replace_other_with_null(data_emt)
data_wire = replace_other_with_null(data_wire)
data_eft = replace_other_with_null(data_eft)

print(data_abm.isnull().sum()/len(data_abm))
print(data_card.isnull().sum()/len(data_card))
print(data_eft.isnull().sum()/len(data_eft))
print(data_emt.isnull().sum()/len(data_emt))
print(data_kyc.isnull().sum()/len(data_kyc))
print(data_wire.isnull().sum()/len(data_wire))
print(data_kyc_industry_codes.isnull().sum()/len(data_kyc_industry_codes))





abm_id              0.000000
customer_id         0.000000
amount_cad          0.000000
debit_credit        0.000000
cash_indicator      0.000000
country             0.185506
province            0.416227
city                0.416227
transaction_date    0.000000
transaction_time    0.000000
dtype: float64
card_trxn_id         0.000000
customer_id          0.000000
amount_cad           0.000000
debit_credit         0.000000
merchant_category    0.173314
ecommerce_ind        0.000000
country              0.378131
province             0.390205
city                 0.517976
transaction_date     0.000000
transaction_time     0.000000
dtype: float64
eft_id              0.0
customer_id         0.0
amount_cad          0.0
debit_credit        0.0
transaction_date    0.0
transaction_time    0.0
dtype: float64
emt_id              0.0
customer_id         0.0
amount_cad          0.0
debit_credit        0.0
transaction_date    0.0
transaction_time    0.0
dtype: float64
customer_id         0.000000
cou

We observed that the count for the null value for the city is relatively small, we decided to drop all rows where 'city' entry is null

In [ ]:
# Function to encode categorical columns and perform multiple imputation
def impute_missing_values(dataset, columns_to_impute):
    # Convert categorical columns to numerical using category codes
    data_for_imputing = dataset.copy()
    dataset_categorical ={} 
    for column in columns_to_impute:
        dataset_categorical[column] = data_for_imputing[column].astype('category').cat
        dataset[column] =dataset_categorical.codes
        dataset[column].replace(-1, np.nan, inplace=True)  # Ensure NaN for missing values

    # Initialize the kernel for miceforest
    kernel = mf.ImputationKernel(
        dataset[columns_to_impute],
        save_all_iterations=True,
        random_state=42
    )

    # Perform multiple imputations
    kernel.mice(5)  # Perform 5 multiple imputation iterations
    imputed_data = kernel.complete_data(dataset=0)
    # Convert numerical encodings back to original categories
    for column in columns_to_impute:
        # Ensure the column is treated as categorical
        dataset[column] = pd.Categorical.from_codes(
            codes=imputed_data[column].round().astype(int),  # Ensure integer codes
            categories=dataset_categorical.categories  # Use original categories
        )
    return dataset


# Specify columns to impute
columns_to_impute = ['city', 'province', 'country']

# Perform imputation on each dataset
print("Imputing data_abm...")
data_abm_imputed = impute_missing_values(data_abm, columns_to_impute)

print("Imputing data_card...")
data_card_imputed = impute_missing_values(data_card, columns_to_impute)

print("Imputing data_kyc...")
data_kyc_imputed = impute_missing_values(data_kyc, columns_to_impute)


Imputing data_abm...


AttributeError: 'DataFrame' object has no attribute 'cat'

In [33]:
print(data_abm.head())

                 abm_id       customer_id  amount_cad debit_credit  \
0  ABM00000000000000006  SYNCID0000000014       25.41       credit   
1  ABM00000000000000008  SYNCID0000000034      238.17        debit   
2  ABM00000000000000009  SYNCID0000000034     1655.43       credit   
3  ABM00000000000000010  SYNCID0000000034      620.69       credit   
4  ABM00000000000000011  SYNCID0000000034      323.70        debit   

   cash_indicator country province  city transaction_date transaction_time  
0           False     0.0      7.0  50.0       2022-11-16         17:37:41  
1            True     0.0      5.0  63.0       2022-11-18         10:22:59  
2           False     0.0      0.0  18.0       2022-12-29         11:56:08  
3            True     0.0      0.0  10.0       2023-01-22         16:48:12  
4            True     0.0      5.0  74.0       2022-11-14         13:24:45  


## Correlation analysis